In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "utility"
    cd(joinpath(@__DIR__, "../../"))
    Pkg.activate("Project.toml")
end

using MorphoMol
using JLD2
using LinearAlgebra
using Rotations
using Printf

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`


In [7]:
function get_aggregate_simulation_evaluation(folder)
    mol_type = "6r7m"
    assembly_successes = 0
    total_simulations = 0
    id_index = 1
    steps_from_init_to_success = Vector{Int}([])
    steps_from_last_max_to_success = Vector{Int}([])
    acceptance_rates = Vector{Float64}([])

    for file in readdir(folder)
        if split(file, ".")[end] == "jld2"
            try
                @load "$folder$file" input output

                E_min_index = argmin(output["Es"])
                E_max_index = argmax(output["Es"])

                # Crude test to exclude runs that remained dispersed
                if maximum(output["Es"]) - minimum(output["Es"]) < 10.0
                    println("E_diff = $(maximum(output["Es"]) - minimum(output["Es"]))")
                    continue
                end

                exp_template_centers = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["template_centers"]
                exp_state = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["state"]
                theta = MorphoMol.Utilities.average_offset_distance(exp_template_centers, input["template_centers"], exp_state, output["states"][argmin(output["Es"])])
                push!(acceptance_rates, output["αs"][end])

                E_max = maximum(output["Es"])
                last_E_disp_index = E_max_index > E_min_index ? 1 : findall(x->x≈E_max, output["Es"][1:E_min_index])[end]
    
                total_simulations += 1
                if theta < 2.0
                    assembly_successes += 1
                    push!(steps_from_init_to_success, E_min_index)
                    push!(steps_from_last_max_to_success, E_min_index - last_E_disp_index)
                end
            catch e
                println("Error in $file")
            end
        end
    end
    return total_simulations, assembly_successes/total_simulations, sum(steps_from_init_to_success)/length(steps_from_init_to_success), sum(steps_from_last_max_to_success)/length(steps_from_init_to_success), sum(acceptance_rates) / length(acceptance_rates)
end

get_aggregate_simulation_evaluation (generic function with 1 method)

In [3]:
#folder = "../../Data/collected_simulation_results/rwm_ma_2_6r7m/"
folder = "../../Data/hpc_out/rwm_ma_2_6r7m/"
ts, sr, sfits, sfmts, aar = get_aggregate_simulation_evaluation(folder)
println("Morphometric Approach")
@printf("Number of simulations: %.0f, Success ratio: %.3f\nSteps from init to success: %.0f, Steps from last max to success: %.0f\nAverage acceptance rate: %.2f", ts, sr, sfits, sfmts, aar)

Error in 76_rwm_ma_2_6r7m.jld2
Morphometric Approach
Number of simulations: 98, Success ratio: 0.051
Steps from init to success: 5307, Steps from last max to success: 3080
Average acceptance rate: 0.24

In [4]:
folder = "../../Data/hpc_out/rwm_wp_2_6r7m/"
ts, sr, sfits, sfmts, aar = get_aggregate_simulation_evaluation(folder)
println("Morphometric Approach with Persistence")
@printf("Number of simulations: %.0f, Success ratio: %.3f\nSteps from init to success: %.0f, Steps from last max to success: %.0f\nAverage acceptance rate: %.2f", ts, sr, sfits, sfmts, aar)

Morphometric Approach with Persistence
Number of simulations: 99, Success ratio: 0.131
Steps from init to success: 4371, Steps from last max to success: 3943
Average acceptance rate: 0.19

In [5]:
folder = "../../Data/hpc_out/rwm_op_2_6r7m/"
ts, sr, sfits, sfmts, aar = get_aggregate_simulation_evaluation(folder)
println("Only Persistence")
@printf("Number of simulations: %.0f, Success ratio: %.3f\nSteps from init to success: %.0f, Steps from last max to success: %.0f\nAverage acceptance rate: %.2f", ts, sr, sfits, sfmts, aar)

Error in 75_rwm_op_2_6r7m.jld2
Error in 84_rwm_op_2_6r7m.jld2
Only Persistence
Number of simulations: 97, Success ratio: 0.000
Steps from init to success: NaN, Steps from last max to success: NaN
Average acceptance rate: 0.24

In [6]:
folder = "../../Data/hpc_out/rwm_wip_2_6r7m/"
ts, sr, sfits, sfmts = get_aggregate_simulation_evaluation(folder)
println("Morphometric Approach with Interface Persistence")
@printf("Number of simulations: %.0f, Success ratio: %.3f, Steps from init to success: %.0f, Steps from last max to success: %.0f\n", ts, sr, sfits, sfmts)

E_diff = 9.873233621632608
E_diff = 5.472242272278891
E_diff = 7.146086029048092
E_diff = 8.55306337546972
Error in 70_rwm_wip_2_6r7m.jld2
E_diff = 0.22657185415118875
Error in 8_rwm_wip_2_6r7m.jld2
Error in 91_rwm_wip_2_6r7m.jld2
Morphometric Approach with Interface Persistence
Number of simulations: 84, Success ratio: 0.000, Steps from init to success: NaN, Steps from last max to success: NaN
